In [1]:
import matplotlib.pyplot as plt
import skimage
import imageio
from tqdm import tqdm

%matplotlib inline


/Users/pascal/opt/anaconda3/envs/first-order-model/lib/python3.7/site-packages/dask/config.py:129: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [ ]:
def plot_images(images):
    fig = plt.figure(fig_size=(15, 15))
    n_col = 4
    n_row = math.ceil(len(images) %% n_col)
    for i, im in enumerate(images):
        plt.subplot(n_col, n_row, i+1)
        plt.imshow(im)
    plt.show()


# Quellen

- Siarohin, A & Lathuilière, S & Tulyakov, S & Ricci, E & Sebe, N 2019 'First Order Motion Model for Image Animation', _Conference on Neural Information Processing Systems (NeurIPS)_, Dezember, 2019

- Bulat, A & Tzimiropoulos, G 2017 'How far are we from solving the 2D & 3D Face Alignment problem? (and a dataset of 230,000 3D facial landmarks', _International Conference on Computer Vision_, 2017

# Data Preprocessing
In einem ersten Schritt müssen wir unsere Daten vorbereiten. Dazu zählt das Video sowie das zu animierende Bild.

### Video

Ein Video besteht aus mehreren Frames:

In [ ]:
path_video = "video.mp4"
video = imageio.get_reader(path_video)

frames = [video[i] for i in range(0,5,20)]
plot_images(frames)

### Bounding Boxes

Für unser Fake-Video konzentrieren wir uns nur auf das Gesicht. Daher wollen wir das Video entsprechend zuschneiden.
Wir suchen in den einzelnen Frames nach dem Gesicht und zeichnen eine Bounding Box darum:

In [ ]:
device = 'cpu'
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device=device)

def extract_bbox(frame, fa):
    if max(frame.shape[0], frame.shape[1]) > 640:
        scale_factor =  max(frame.shape[0], frame.shape[1]) / 640.0
        frame = resize(frame, (int(frame.shape[0] / scale_factor), int(frame.shape[1] / scale_factor)))
        frame = img_as_ubyte(frame)
    else:
        scale_factor = 1
    frame = frame[..., :3]
    bboxes = fa.face_detector.detect_from_image(frame[..., ::-1])
    if len(bboxes) == 0:
        return []
    return np.array(bboxes)[:, :-1] * scale_factor

for frame in frames:
    bboxes = extract_bbox(frame, fa)
    print(bbox)

### Video zuschneiden

Nachdem wir die Bounding-Boxen für jedes Frame gefunden haben, schneiden wir das Video entsprechend zu.

### Zugeschnittenes Video
Das neue Video sieht nur wie folgt aus:

### Bild laden

Neben dem Video benötigen wir auch ein Bild von Guy Parmelin

# Deep Fake

Nachdem die Daten vorbereitet sind, können wir nun unser Fake Video erzeugen. Dies funktioniert mit zwei Modulen


1. Modul zur Schätzung der Bewegung
2. Modul zur Bilderzeugung


### Modul zur Bewegungsabschätzung
- Ein neuronales Netzwerk lernt in Bildern Schlüsselpunkte zu erkennen.

-> Bild Schlüsselpunkte von treibendem Video sowie der Bilddatei

- Wenn die Schlüsselpunkte aus den Videoframes mit dem Bild verglichen werden, können die Bewegungen abgeschätzt werden

-> Bild einfügen als Beispiel

- Die Bewegungen werden dann kodiert an das Modul zur Bilderzeugung übergeben


### Modul zur Bilderzeugung

- Das zweite Modul erzeugt das Bild. Dazu schätzt es ab, welche Bewegungen durch Verzerren des Bildes bewirkt werden können und wo das Bild "übermalt" werden soll
- Anschliessend werden die angepassten Bilder wieder zu einem Video zusammengefügt



### Training

Ein neuronales Netzwerk erlernt wie es diese Aufgaben vollautomatisch erfüllen kann. Dieses Erlernen erfordert jedoch viel Rechenleistung und dauert lange.

Deshalb verwenden wir in unserem Beispiel ein bereits trainiertes Netzwerk. Deshalb können wir direkt ein Fake-Video erzeugen.

### Videos zusammenfügen

Damit wir das Resultat besser sehen, fügen wir das originale sowie das erzeugte Video zusammen.

### Resultat

# Weitere Möglichkeiten

### Verschiedene Annimationen:

![Screenshot](../sup-mat/vox-teaser.gif)

![Screenshot](../sup-mat/fashion-teaser.gif)

![Screenshot](../sup-mat/mgif-teaser.gif)


### Face Swap

![Screenshot](../sup-mat/face-swap.gif)